# tf.dataを使ったテキストの読み込み

このチュートリアルでは、`tf.data.TextLineDataset`を使ってテキストファイルからサンプルを読み込む方法を例示する。
`TextLineDataset`は、テキストファイルからデータセットを作成するために設計されている。この中では、元のテキストファイルの一行一行がサンプルになっている。
これは、（例えば、詩やエラーログのような）基本的に行ベースのテキストデータを扱うのに便利。

このチュートリアルでは、同じ作品であるホーマーのイリアッドの異なる３つの英語翻訳版を使い、テキスト１行から翻訳者を特定するモデルを構築する。

## 設定

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import tensorflow_datasets as tfds
import os

3つの翻訳のテキストは次のとおり。

- [William Cowper](https://en.wikipedia.org/wiki/William_Cowper) - [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt)
- [Edward, Earl of Derby](https://en.wikipedia.org/wiki/Edward_Smith-Stanley,_14th_Earl_of_Derby) - [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt)
- [Samuel Butler](https://en.wikipedia.org/wiki/Samuel_Butler_%28novelist%29) - [text](https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt)

このチュートリアルで使われているテキストファイルは、ヘッダ、フッタ、行番号、章のタイトルの削除など、いくつかの典型的な前処理を行ったもの。前処理後のファイルをダウンロードする。

In [4]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
    text_dir = tf.keras.utils.get_file(name, origin=DIRECTORY_URL+name)

parent_dir = os.path.dirname(text_dir)

parent_dir

'C:\\Users\\ognek\\.keras\\datasets'

## テキストをデータセットに読み込む

ファイルをイテレートし、それぞれを別々のデータセットに読み込む。

サンプルはそれぞれにラベル付けが必要なので、ラベル付け関数を適用するために`tf.data.Dataset.map`を使う。
このメソッドは、データセット無いのすべてのサンプルをイテレートし、(`example, label`)というペアを返す。

In [6]:
def labeler(example, index):
    return example, tf.cast(index, tf.int64)

labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
    print(i)
    lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    labeled_data_sets.append(labeled_dataset)

0
1
2


ラベル付けの終わったデータセットを結合して一つのデータセットにし、シャッフルする。

In [8]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

In [10]:
all_labeled_data = labeled_data_sets[0]
for labeled_dtaset in labeled_data_sets[1:]:
    all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False
)

`tf.data.Dtaset.take`と`print`を使って、`(example, label)`のペアがどのようなものかを見ることができる。
`numpy`プロパティがそれぞれのテンソルの値を示す。

In [12]:
for ex in all_labeled_data.take(5):
    print(ex)

(<tf.Tensor: shape=(), dtype=string, numpy=b'Should wage before you the wide-wasting war.'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'necks of the horses Then Juno put her steeds under the yoke, eager for'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Confusion seized his brain; his noble limbs'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'Should any of the everlasting Gods'>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'hold your ground here, and go about among the host to rally them in'>, <tf.Tensor: shape=(), dtype=int64, numpy=2>)


## テキスト行を数字にエンコードする

機械学習モデルが扱うのは単語ではなく数字であるため、文字列は数字のリストに変換する必要がある。
このため、一意の単語を一意の数字にマッピングする。

### ボキャブラリーの構築

まず最初に、テキストをトークン化し、個々の一意な単語の集まりとして、ボキャブラリーを構築します。
これを行うには、TensorFlowやPythonを使ういくつかの方法があります。
ここでは次のようにする。

1. 各サンプルの`numpy`値をイテレートする。
2. `tfds.feature.text.Tokenizer`を使って、それをトークンに分割する。
3. 重複を排除するため、トークンをPythonの集合に集約する。
4. あとで使用するため、ボキャブラリーのサイズを取得する。

In [14]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

14436

### サンプルをエンコードする

`vocabulary_set`を`tfds.features.text.TokenTextEncoder`に渡してエンコーダーを作成する。
エンコーダーの`encode`メソッドは、テキストを文字列に引数にとり、整数のリストを返す。

In [16]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [18]:
# 1行だけに適用してみて、出力がどの様になるか確認する
# テキストが数値に置き換わっていることを確認すること。
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)
encoded_example = encoder.encode(example_text)
print(encoded_example)

# 念の為単語の数が同じ担っていることを確認 (example_textはbyte型なのでstr型に変換してからsplit
print(type(example_text))
print(len(example_text.decode("utf-8").split(' ')))
print(len(encoded_example))

b'Should wage before you the wide-wasting war.'
[9667, 11431, 5190, 12319, 10942, 12689, 3648, 13604]
<class 'bytes'>
7
8


次に、このエンコーダーを`tf.py_function`でラッピングして、データセットの`map`メソッドに渡し、データセットに適用する。

In [20]:
def encode(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

def encode_map_fn(text, label):
    # py_func は返してくるテンソルのshapeをsetしない
    encoded_text, label = tf.py_function(encode,
                                         inp=[text, label],
                                         Tout=(tf.int64, tf.int64))
    # `tf.data.Dataset`はすべての要素がshapeを持っているときに（最良に？）動作するため、
    # shapeを手動で与えておく
    encoded_text.set_shape([None])
    label.set_shape([])

    return encoded_text, label

all_encoded_data = all_labeled_data.map(encode_map_fn)

## データセットを、テスト用と訓練用のバッチに分割する
`tf.data.Dataset.take`とtf.data.Dataset.skip`を使って、小さなテスト用データセットを、より大きな訓練用セットを作成する。  

モデルに渡す前に、データセットをバッチ化する必要がある。通常、バッチの中のサンプルは、同じサイズと形状である必要がある。しかし、これらのデータセットの中にはのサンプルはすべて同じサイズではない。
具体的には、テキストの各行の単語数は異なっている。
このため、（`batch`の代わりに）`tf.data.Dataset.padded_batch`メソッドを使ってサンプルを同じサイズにパディングする。

In [22]:
# チュートリアルのとおりにやるとエラーが出る。
# padded_batchの第2引数を与えないとエラーが出てしまうバグあるようなので追加。
# https://github.com/tensorflow/tensorflow/issues/36308
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE,  ([None],()))

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE,  ([None],()))

この段階で、`test_data`と`train_data`は、(`example, label`)というペアのコレクションではなく、バッチのコレクションになっている。
それぞれのバッチは、(*たくさんのサンプル, たくさんのラベル*)という配列のペアになっている。

In [24]:
sample_text, sample_labels = next(iter(test_data))
sample_text[0], sample_labels[0]

(<tf.Tensor: shape=(16,), dtype=int64, numpy=
 array([ 9667, 11431,  5190, 12319, 10942, 12689,  3648, 13604,     0,
            0,     0,     0,     0,     0,     0,     0], dtype=int64)>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>)

（ゼロをパディングに使用したことで）新しいトークン番号を1つ導入したので、ボキャブラリーサイズは1つ増えている。

In [26]:
vocab_size += 1

## モデルを構築する

最初の層は、整数表現を密なベクトル埋め込みに変換する。詳細は
[単語埋め込み](https://www.tensorflow.org/tutorials/text/word_embeddings?hl=ja)
のチュートリアルを参照。

次の層は、Long Short-Term Memory層(LSTM層)とする。この層により、モデルは単語を他の単語の文脈の中で解釈する。
LSTMのBidirectionalラッパーにより、データポイントを、その前とその後のデータポイントとの関連で学習することができる。

続いての層は、1つ以上の全結合層がいくつかあるとして、最終層が出力層となる。最終層はラベルすべての確率を生成する。
最も確率の高いラベルが、モデルが余録するサンプルのラベルとなる。

In [28]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, 64))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

# 1つ以上の全結合層を追加。今回は64unitの層を2つ入れている。
# 別の値をいれて実験してもよい
for units in [64, 64]:
    model.add(tf.keras.layers.Dense(units, activation="relu"))

# 出力層 最初の引数はラベルの数 (= 3人のうちどの翻訳者かを推定するので、3)
model.add(tf.keras.layers.Dense(3, activation="softmax"))

最後似モデルをコンパイルする。softmaxによるカテゴリー分類モデルでは、損失関数として`sparse_categorical_crossentropy`を使用する。
他のoptimizerを使うこともできるが、optimizerには`adam`がよく使われる。

In [30]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

## モデルを訓練する

このモデルをこのデータに適用すると、約83%のまともな結果が得られる。
（実際に動かしてみると99%も出てしまう。。高すぎ？）

In [32]:
model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3
601/601 [==============================] - 16s 26ms/step - loss: 0.0041 - accuracy: 0.9986 - val_loss: 0.0439 - val_accuracy: 0.9880
Epoch 2/3
601/601 [==============================] - 14s 24ms/step - loss: 0.0025 - accuracy: 0.9992 - val_loss: 0.0404 - val_accuracy: 0.9912
Epoch 3/3
601/601 [==============================] - 14s 23ms/step - loss: 0.0016 - accuracy: 0.9996 - val_loss: 0.0302 - val_accuracy: 0.9930


In [34]:
eval_loss, eval_acc = model.evaluate(test_data)
print("\nEval loss: {}, Eval accuracy: {}".format(eval_loss, eval_acc))

79/Unknown - 2s 23ms/step - loss: 0.0302 - accuracy: 0.9930
Eval loss: 0.030177595910940967, Eval accuracy: 0.9929999709129333
